# 機械学習（実践講義4）：Neural Network

## 課題4.1 多層パーセプトロン学習プログラムの作成

ニューラルネットワークによる学習を実現するプログラムを作成せよ． ここでは，ニューラルネットワークは，
* 入力層
* 中間層（隠れ層）
* 出力層
からなるものとする．

このとき，特徴ベクトル（拡張特徴ベクトル）の要素数，パターン数，およびクラス数は任意の数を設定できるように作ること．


**誤差逆伝播法の手順**

1. 中間層と出力層の初期重みを決定する．
1. 入力層から順番に，各ユニットの出力を計算していく．（中間層の出力をg1, 出力層の出力をg2とすると）
  * 中間層のユニット数分ループしながらg1[j]の値を計算．
  * 出力層のユニット数分ループしながらg2[k]の値を計算．
1. 教科書P.46 式(3.68)やを参考に重みを修正．（入力層→中間層の重みをw1, 中間層→出力層の重みをw2とすると）
  * ε[j]を計算． w2[j][k]の値を修正．
  * 更新前のw2を用いてε[j]を計算． w1[i][j]の値を修正．
1. 教科書P.47-48ページを参考に，収束の判定を行う．
1. 手順２に戻る

**実験用データ**

完成したプログラムに，以下のデータを入力し，正しく学習することを確認せよ．
ただし，
$\rho=1$
とする．

|パターン|値|クラス|
| ---- | ---- | ---- |
|パターン1|(1,1)|ω1|
|パターン2|(2,1)|ω1|
|パターン3|(1,3)|ω2|
|パターン4|(2,4)|ω2|
|パターン5|(4,3)|ω3|
|パターン6|(4,2)|ω3|

この数値データを値とクラスに分けてファイルに保存する．

* data3.csv
```
1, 1
2, 1
1, 3
2, 4
4, 3
4, 2
```
* label3.txt
```
1,0,0
1,0,0
0,1,0
0,1,0
0,0,1
0,0,1
```

**ヒント**

***拡張ベクトル化***

```python
x1 = np.hstack((x[p], [1]))
```

***重みの初期化***

入力層2，中間層4，出力層3の場合
```python
w1 = np.random.rand(4, 2+1)  # 中間層の数 x 拡張ベクトルの長さ
w2 = np.random.rand(3, 4+1)  # 出力層の数 x 中間の拡張ベクトルの長さ(中間層+1)
```

***説明図***
![プログラムの変数との対応](figure_bp.png)


In [231]:
# 必要なモジュールの読み込み
import numpy as np

In [232]:
# 学習率
rho = 1.0

In [233]:
# データの読み込み

x = np.loadtxt("data3.csv", delimiter=",")
y = np.loadtxt("label3.txt", delimiter=',', dtype=int)

In [234]:
x, y

(array([[ 1.,  1.],
        [ 2.,  1.],
        [ 1.,  3.],
        [ 2.,  4.],
        [ 4.,  3.],
        [ 4.,  2.]]), array([[1, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 1]]))

In [235]:
w1 = np.random.randn(4, 3)
w2 = np.random.randn(3, 5)

tau = 0.01

def sigmoid(s):
    return 1/(1+np.exp(-s))
       

while True:
    loss = 0
    
    for p in range(len(x)):
        # xを拡張ベクトル化
        x1 = np.hstack((x[p], [1]))
        
    
     
        # 1層目（x1とw1を使って計算）
        layer1 = np.dot(x1, w1.T)
        g1 = sigmoid(layer1)
        
     
      
        # 1層目の出力を拡張ベクトル化
        g11 = np.hstack((g1, [1]))
        
        
        # 2層目（g11とw2を使って計算）
        layer2 = np.dot(g11, w2.T)
        g2 = sigmoid(layer2)
        
        

        # 誤差の合計を求めるため， x[p]に対する誤差を loss に加算 
        loss += ((g2-y[p])**2)
        #print(loss)
 
        # eps1を計算するため，w2の更新前に，w2を一旦保存しておく（np.copyによりデータをコピー)
        w2_old = np.copy(w2)

        # g2とy[p]を使ってeps2[0], eps2[1], eps2[2]を計算
        eps2 = (g2-y[p])*g2*(1 - g2)
        
        
    
        # w2を更新
        for k in range(w2.shape[0]):
            for j in range(w2.shape[1]):
                w2[k, j] -= rho * np.dot(eps2[k],g11[j])
                
       
        # eps2 と w2_old と g1を使ってeps1[0], eps1[1], eps1[2], eps1[3]を計算
        eps1 = g1*(1 - g1)
        for j in range(w1.shape[0]):
            tmp = 0
            for k in range(w2.shape[0]):
                # ここで，eps2とw2_oldを利用
                tmp += np.dot(eps2[k], w2_old[k,j])
            eps1[j] *= tmp
                
        # w1を更新
        for j in range(w1.shape[0]):
            for i in range(w1.shape[1]):
                w1[j, i] -= rho * np.dot(eps1[j], g11[i])
        
    # 誤差の平均 loss/データ数 が tau 以下ならループ終了
    #print(loss /len(x))
    #if loss / len(x) < tau:
        #break
        
    flag = False
    for b in range(len(loss)):
        #print(loss[int(b)] / int(len(x)))
        if loss[int(b)] / int(len(x)) < tau:
            flag = True
    if flag == True:
        #print(w1, w2) 
        break


In [236]:
for p in range(len(x)):
        # xを拡張ベクトル化
        x1 = np.hstack((x[p], [1]))
        
    
     
        # 1層目（x1とw1を使って計算）
        layer1 = np.dot(x1, w1.T)
        g1 = sigmoid(layer1)
        
     
      
        # 1層目の出力を拡張ベクトル化
        g11 = np.hstack((g1, [1]))
        
        
        # 2層目（g11とw2を使って計算）
        layer2 = np.dot(g11, w2.T)
        g2 = sigmoid(layer2)

        print(g2)
        result = np.argmax(g2)
        print(result)

[ 0.51160276  0.15770685  0.23830848]
0
[ 0.43472456  0.04681183  0.56422654]
2
[ 0.08486657  0.88496244  0.05543941]
1
[ 0.07275264  0.90560828  0.05204636]
1
[ 0.31845623  0.05059283  0.64455955]
2
[ 0.31309137  0.02074666  0.79876293]
2


## 課題4.2

これまでの課題で用いた，アヤメのデータセット（iris_dataset）に対し，<br/>
今回実装したニューラルネットワークを用いて認識し，認識率を計算せよ．

以前実装した，パーセプトロンと比べて認識率がどうなるか調査し，考察せよ．

In [237]:
# 必要なモジュールの読み込み
import numpy as np

In [238]:
# 学習率
rho = 1.0

In [239]:
from sklearn.datasets import load_iris
iris = load_iris()
iris.data

array([[ 5.1,  3.5,  1.4,  0.2],
       [ 4.9,  3. ,  1.4,  0.2],
       [ 4.7,  3.2,  1.3,  0.2],
       [ 4.6,  3.1,  1.5,  0.2],
       [ 5. ,  3.6,  1.4,  0.2],
       [ 5.4,  3.9,  1.7,  0.4],
       [ 4.6,  3.4,  1.4,  0.3],
       [ 5. ,  3.4,  1.5,  0.2],
       [ 4.4,  2.9,  1.4,  0.2],
       [ 4.9,  3.1,  1.5,  0.1],
       [ 5.4,  3.7,  1.5,  0.2],
       [ 4.8,  3.4,  1.6,  0.2],
       [ 4.8,  3. ,  1.4,  0.1],
       [ 4.3,  3. ,  1.1,  0.1],
       [ 5.8,  4. ,  1.2,  0.2],
       [ 5.7,  4.4,  1.5,  0.4],
       [ 5.4,  3.9,  1.3,  0.4],
       [ 5.1,  3.5,  1.4,  0.3],
       [ 5.7,  3.8,  1.7,  0.3],
       [ 5.1,  3.8,  1.5,  0.3],
       [ 5.4,  3.4,  1.7,  0.2],
       [ 5.1,  3.7,  1.5,  0.4],
       [ 4.6,  3.6,  1. ,  0.2],
       [ 5.1,  3.3,  1.7,  0.5],
       [ 4.8,  3.4,  1.9,  0.2],
       [ 5. ,  3. ,  1.6,  0.2],
       [ 5. ,  3.4,  1.6,  0.4],
       [ 5.2,  3.5,  1.5,  0.2],
       [ 5.2,  3.4,  1.4,  0.2],
       [ 4.7,  3.2,  1.6,  0.2],
       [ 4

In [240]:
iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [241]:
target = []
for i in range(len(iris.target)):
    if iris.target[i] == 0:
        target.append([1,0,0])
    elif i == 1:
        target.append([0,1,0])
    else:
        target.append([0,0,1])
print(target)

[[1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1]

In [242]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y=train_test_split(iris.data, target, test_size=0.8, random_state=720)

In [243]:
train_X

array([[ 7. ,  3.2,  4.7,  1.4],
       [ 6.7,  3.3,  5.7,  2.1],
       [ 4.8,  3.4,  1.6,  0.2],
       [ 6.4,  2.9,  4.3,  1.3],
       [ 6.7,  3. ,  5. ,  1.7],
       [ 6. ,  2.2,  5. ,  1.5],
       [ 4.4,  3. ,  1.3,  0.2],
       [ 6.8,  2.8,  4.8,  1.4],
       [ 6. ,  3.4,  4.5,  1.6],
       [ 5.5,  4.2,  1.4,  0.2],
       [ 5.5,  2.5,  4. ,  1.3],
       [ 6.7,  3. ,  5.2,  2.3],
       [ 6.2,  3.4,  5.4,  2.3],
       [ 5.1,  3.7,  1.5,  0.4],
       [ 6.1,  2.8,  4.7,  1.2],
       [ 6.3,  2.8,  5.1,  1.5],
       [ 4.9,  3.1,  1.5,  0.1],
       [ 5.9,  3. ,  5.1,  1.8],
       [ 5.9,  3.2,  4.8,  1.8],
       [ 7.3,  2.9,  6.3,  1.8],
       [ 6.7,  3.1,  4.7,  1.5],
       [ 6.1,  3. ,  4.6,  1.4],
       [ 5. ,  3.6,  1.4,  0.2],
       [ 6.2,  2.8,  4.8,  1.8],
       [ 4.5,  2.3,  1.3,  0.3],
       [ 6.9,  3.1,  5.4,  2.1],
       [ 5.4,  3. ,  4.5,  1.5],
       [ 5.5,  2.3,  4. ,  1.3],
       [ 5. ,  2. ,  3.5,  1. ],
       [ 6.5,  3. ,  5.5,  1.8]])

In [244]:
test_X

array([[ 5.8,  2.7,  3.9,  1.2],
       [ 4.9,  3.1,  1.5,  0.1],
       [ 4.7,  3.2,  1.3,  0.2],
       [ 6. ,  3. ,  4.8,  1.8],
       [ 6.4,  2.8,  5.6,  2.1],
       [ 7.7,  3. ,  6.1,  2.3],
       [ 7.7,  2.8,  6.7,  2. ],
       [ 5.6,  2.5,  3.9,  1.1],
       [ 5.7,  2.8,  4.5,  1.3],
       [ 6. ,  2.2,  4. ,  1. ],
       [ 5.5,  3.5,  1.3,  0.2],
       [ 5.6,  2.7,  4.2,  1.3],
       [ 5. ,  3.2,  1.2,  0.2],
       [ 5.2,  3.4,  1.4,  0.2],
       [ 5.9,  3. ,  4.2,  1.5],
       [ 6.5,  3. ,  5.8,  2.2],
       [ 6.7,  3.3,  5.7,  2.5],
       [ 6.4,  3.2,  5.3,  2.3],
       [ 6. ,  2.9,  4.5,  1.5],
       [ 4.3,  3. ,  1.1,  0.1],
       [ 5.5,  2.4,  3.8,  1.1],
       [ 4.8,  3.4,  1.9,  0.2],
       [ 4.8,  3. ,  1.4,  0.3],
       [ 7.9,  3.8,  6.4,  2. ],
       [ 4.6,  3.2,  1.4,  0.2],
       [ 7.7,  3.8,  6.7,  2.2],
       [ 6.8,  3. ,  5.5,  2.1],
       [ 4.9,  2.5,  4.5,  1.7],
       [ 5.2,  2.7,  3.9,  1.4],
       [ 5.1,  2.5,  3. ,  1.1],
       [ 5

In [245]:
test_y

[[0, 0, 1],
 [1, 0, 0],
 [1, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [1, 0, 0],
 [0, 0, 1],
 [1, 0, 0],
 [1, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [1, 0, 0],
 [0, 0, 1],
 [1, 0, 0],
 [1, 0, 0],
 [0, 0, 1],
 [1, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [1, 0, 0],
 [0, 0, 1],
 [1, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [1, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [1, 0, 0],
 [1, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 0, 1],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [1, 0, 0],
 [1, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [1, 0, 0],
 [0, 0, 1],
 [0,

In [255]:
w1 = np.random.rand(5, 4+1)  # 中間層の数 x 拡張ベクトルの長さ
w2 = np.random.rand(3, 5+1)  # 出力層の数 x 中間の拡張ベクトルの長さ(中間層+1)
tau = 0.01

def sigmoid(s):
    return 1/(1+np.exp(-s))
       
a = 0
#while a < 2000:
while True:
    loss = 0
    a += 1
    
    for p in range(len(train_X)):
        print("p", p)
        # xを拡張ベクトル化
        x1 = np.hstack((train_X[p], [1]))
        
        # 1層目（x1とw1を使って計算）
        layer1 = np.dot(x1, w1.T)
        g1 = sigmoid(layer1)
        
     
      
        # 1層目の出力を拡張ベクトル化
        g11 = np.hstack((g1, [1]))
        
        
        # 2層目（g11とw2を使って計算）
        layer2 = np.dot(g11, w2.T)
        g2 = sigmoid(layer2)
        print(g2)
        

        # 誤差の合計を求めるため， x[p]に対する誤差を loss に加算 
        loss += (g2-train_y[p])**2
        print("loss: " + str(loss))
       
 
        # eps1を計算するため，w2の更新前に，w2を一旦保存しておく（np.copyによりデータをコピー)
        w2_old = np.copy(w2)

        # g2とy[p]を使ってeps2[0], eps2[1], eps2[2]を計算
        eps2 = (g2-train_y[p])*g2*(1 - g2)
        
        
    
        # w2を更新
        for k in range(w2.shape[0]):
            for j in range(w2.shape[1]):
                w2[k, j] -= rho * np.dot(eps2[k],g11[j])
                
       
        # eps2 と w2_old と g1を使ってeps1[0], eps1[1], eps1[2], eps1[3]を計算
        eps1 = g1*(1 - g1)
        for j in range(w1.shape[0]):
            tmp = 0
            for k in range(w2.shape[0]):
                # ここで，eps2とw2_oldを利用
                tmp += np.dot(eps2[k], w2_old[k,j])
            eps1[j] *= tmp
                
        # w1を更新
        for j in range(w1.shape[0]):
            for i in range(w1.shape[1]):
                w1[j, i] -= rho * np.dot(eps1[j], g11[i])
        
    # 誤差の平均 loss/データ数 が tau 以下ならループ終了
    #print(loss /len(x))
    #if loss / len(x) < tau:
        #break
    
    #flag = False    
    for b in range(len(loss)):
        print(a,loss[int(b)] / int(len(train_X)))
        if loss[int(b)] / int(len(train_X)) < tau:
            flag = False
    if flag == False:
        break
        
print(w1,w2)        

p 0
[ 0.91173739  0.94195682  0.93179951]
loss: [ 0.83126507  0.88728265  0.00465131]
p 1
[ 0.86939579  0.92261773  0.93347229]
loss: [ 1.58711412  1.73850613  0.00907724]
p 2
[ 0.78698261  0.88888471  0.93356795]
loss: [ 1.63249052  2.52862216  0.88062636]
p 3
[ 0.81973113  0.82711848  0.91076934]
loss: [ 2.30444965  3.21274714  0.88858847]
p 4
[ 0.68763305  0.70200337  0.9142967 ]
loss: [ 2.77728887  3.70555588  0.89593353]
p 5
[ 0.47604319  0.49423275  0.91736839]
loss: [ 3.00390599  3.94982189  0.90276151]
p 6
[ 0.31855622  0.32902923  0.9180669 ]
loss: [ 3.46827162  4.05808213  1.74560834]
p 7
[ 0.51516883  0.23343866  0.88490176]
loss: [ 3.73367054  4.11257573  1.75885594]
p 8
[ 0.32971274  0.19180155  0.89184239]
loss: [ 3.84238104  4.14936357  1.77055401]
p 9
[ 0.24879181  0.17288348  0.89642984]
loss: [ 4.40669479  4.17925226  2.57414047]
p 10
[ 0.42181196  0.1469238   0.84291437]
loss: [ 4.58462012  4.20083887  2.59881636]
p 11
[ 0.28215066  0.13302163  0.8588437 ]
loss: [ 4.

In [256]:
print(w1, w2)
result = []
for p in range(len(test_X)):
        # xを拡張ベクトル化
        x1 = np.hstack((test_X[p], [1]))
        
        # 1層目（x1とw1を使って計算）
        layer1 = np.dot(x1, w1.T)
        g1 = sigmoid(layer1)
        
     
      
        # 1層目の出力を拡張ベクトル化
        g11 = np.hstack((g1, [1]))
        
        
        # 2層目（g11とw2を使って計算）
        layer2 = np.dot(g11, w2.T)
        g2 = sigmoid(layer2)
        
        print(g2)
        result.append(np.argmax(g2))
print(result)
        


[[ 0.44964311  0.40529811  0.6371577   0.91520199  0.70842047]
 [ 0.52875326  0.34553823  0.60931586  0.40847412  0.43250134]
 [ 0.13814495 -0.01364635  0.72576101  0.2281832   0.27154139]
 [ 0.2978353   0.75955959  0.56971889  0.41560819  0.93505653]
 [ 0.6514423   0.29626068  0.51173103  0.88655756  0.72835505]] [[-0.67135293  0.15937013 -0.67305806 -0.05972215 -0.68602167  0.07242336]
 [-0.81805769 -0.00667099 -0.82470173 -0.18226735 -0.23496817 -0.65382293]
 [-0.10070409  0.25190402  0.13977358  0.71394045  0.45338893  0.4245194 ]]
[ 0.13616657  0.06255434  0.86760754]
[ 0.14501514  0.06800131  0.86545103]
[ 0.14646203  0.06890842  0.86511735]
[ 0.13545482  0.06211837  0.86779599]
[ 0.13517059  0.06194452  0.86786516]
[ 0.13504728  0.0618688   0.86789869]
[ 0.13500044  0.06184019  0.86790866]
[ 0.13623838  0.06259855  0.86757409]
[ 0.13571461  0.06227708  0.86772203]
[ 0.13610422  0.06251725  0.86759962]
[ 0.14518283  0.06812353  0.86557873]
[ 0.13592603  0.06240692  0.8676648 ]
[ 

In [257]:
correctness = 0
for n in range(len(test_y)):
    if result[n] == test_y[n]:
        correctness += 1
print(correctness / len(result))
test_y

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()